In [ ]:
import os
hf_token = "HUGGINGFACE TOKEN HERE"
os.environ["HUGGINGFACEHUB_API_TOKEN"]=hf_token
os.environ["HF_TOKEN"]=hf_token
os.environ['HF_HOME'] = 'YOUR_HOME_DIR/.cache/huggingface/'
os.environ['TRANSFOMERS_CACHE'] = 'YOUR_HOME_DIR/.cache/huggingface/'
import transformers

In [ ]:
import os
from pathlib import Path
from tempfile import mkdtemp
from warnings import filterwarnings
from transformers import BitsAndBytesConfig
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import torch
from llama_index.core import Settings
from transformers import AutoModelForCausalLM, AutoTokenizer
from llama_index.llms.huggingface import HuggingFaceLLM
from peft import PeftModel

EMBED_MODEL = HuggingFaceEmbedding(model_name="abhinand/MedEmbed-large-v0.1")
model_id = "meta-llama/Llama-3.1-8B-Instruct"

lora_path="YOUR_HOME_DIR/LLaMA-Factory/saves/llama3-8b/noCOT/lora/epoch10_rank64_lr10e-5"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(model_id,
    device_map="auto",
    #device_map={"": accelerator.process_index},
    token=hf_token,
    quantization_config=quantization_config,
    torch_dtype=torch.float16,
)
model = PeftModel.from_pretrained(model,lora_path)
model = model.merge_and_unload()

#GEN_MODEL = HuggingFaceLLM(model=model,tokenizer_name=model_id,
#    device_map="auto",
#    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
#)

tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)

Settings.tokenzier = tokenizer
#Settings.llm = model

embed_dim = len(EMBED_MODEL.get_text_embedding("hi"))
print("Embed dim:", embed_dim)

In [ ]:
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.readers.docling import DoclingReader
from llama_index.vector_stores.milvus import MilvusVectorStore
from docling.document_converter import DocumentConverter
from llama_index.node_parser.docling import DoclingNodeParser
from docling.chunking import HybridChunker

SOURCE = r"YOUR_HOME_DIR/guideline_edit.md"

reader = DoclingReader()
node_parser = MarkdownNodeParser()
chunker = HybridChunker()

vector_store = MilvusVectorStore(
    uri=str(Path("YOUR_HOME_DIR/datasets/docling_md_vectordb.db")),
    dim=embed_dim,
    overwrite=False,
)
index = VectorStoreIndex.from_documents(
    documents=reader.load_data(SOURCE),
    transformations=[node_parser],
    storage_context=StorageContext.from_defaults(vector_store=vector_store),
    embed_model=EMBED_MODEL,
)

In [ ]:
from html import unescape
QUERY = "Welche Langzeitrisiken bestehen nach chirurgischem Verschluss eines persistierenden Ductus arteriosus (PDA) im Säuglingsalter?"

retriever = index.as_retriever(similarity_top_k=5)
retrieved_docs = retriever.retrieve(QUERY)
sources = [s.get_content(s.metadata) for s in retrieved_docs]
sourcesStr = "\n\n".join(sources)

QUERY = f"### Input:\n{QUERY}\nContext:\n{sourcesStr}\n"

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_new_tokens=1024,
    #max_length=16384,
    do_sample=True,
    temperature=0.6,
    num_return_sequences=1,
    repetition_penalty=1.5,
    eos_token_id=[
        128001,
        128008,
        128009
      ],
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
)

Instruction = "### Instruction:\nYou are a medical QA bot that is tasked to answer questions as accurately as possible given excerpts of a medical guideline. If possible provide short but adequate answers to the given question based on the given context. Avoid repetitions and duplications. Do not add notes or any other information to the output except factually relevant information. If you cannot answer the question with the given information, decline generating the answer."

QUERY = QUERY + "\n\n### Response:"

QUERY = f"<begin_of_text><start_header_id>system<end_header_id>{Instruction}<leot_id><start_header_id>user<end_header_id>{QUERY}<leot_id><start_header_id>assistant<end_header_id>"

# Output and load valid JSON
output_dict = pipeline(QUERY)
output = output_dict[0]["generated_text"][len(QUERY) :]

unescapedOutput = unescape(output)
print("Output:", unescapedOutput)

In [ ]:
query = "How are innocent, functional, and organic heart murmurs defined and differentiated based on their underlying causes?"

retriever = index.as_retriever(similarity_top_k=5)

retrieved_docs = retriever.retrieve(query)
print(retrieved_docs)

sources = [s.get_content(s.metadata) for s in retrieved_docs]
print(sources)

In [ ]:
QUERY = "How are innocent, functional, and organic heart murmurs defined and differentiated based on their underlying causes?"
result = index.as_query_engine(llm=GEN_MODEL, similarity_top_k=5, max_new_tokens=1024).query(QUERY)
print(f"Q: {QUERY}\nA: {result.response.strip()}\n{'-'*50}\nSources:")
display([(n.text, n.metadata) for n in result.source_nodes])